In [35]:
import pandas as pd
from pathlib import Path
from pandas.api.types import is_datetime64_any_dtype as is_datetime
import src.utils as utils
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.core.pylabtools import figsize
import src.data_processing.data as data
from shapely.geometry import Point

In [36]:
X = data.full_trips['Delay time']
full_trips_sorted = data.full_trips.sort_values(by = ['Train number' , 'Date of departure', 'Time of planned departure', ])
full_trips_sorted

,Unnamed: 0,Date of departure,Train number,Relation,Railway operators,Railway of departure,Time of real arrival,Time of real departure,Time of planned arrival,Time of planned departure,Direction of relation,Name of the stop,Railway of arrival,Date of planned arrival,Date of planned departure,Date of real arrival,Date of real departure,Delay time
13572,13572,2020-09-20,10,ICE,SNCB/NMBS,37,2022-10-31 20:36:30,2022-10-31 20:36:30,2022-10-31 20:29:00,2022-10-31 20:29:00,ICE: FRANKFURT(MAIN) HBF -> BRUSSEL-ZUID,hergenrath,37,2020-09-20 00:00:00,2020-09-20,2020-09-20,2020-09-20,450
13575,13575,2020-09-20,10,ICE,SNCB/NMBS,37,2022-10-31 20:47:50,2022-10-31 20:47:50,2022-10-31 20:40:00,2022-10-31 20:40:00,ICE: FRANKFURT(MAIN) HBF -> BRUSSEL-ZUID,chênée,3,2020-09-20 00:00:00,2020-09-20,2020-09-20,2020-09-20,470
14842,14842,2020-09-20,10,ICE,SNCB/NMBS,37,2022-10-31 20:48:43,2022-10-31 20:48:43,2022-10-31 20:41:00,2022-10-31 20:41:00,ICE: FRANKFURT(MAIN) HBF -> BRUSSEL-ZUID,angleur,37,2020-09-20 00:00:00,2020-09-20,2020-09-20,2020-09-20,463
14845,14845,2020-09-20,10,ICE,SNCB/NMBS,36,2022-10-31 20:51:05,2022-10-31 20:52:54,2022-10-31 20:44:00,2022-10-31 20:46:00,ICE: FRANKFURT(MAIN) HBF -> BRUSSEL-ZUID,liège-guillemins,37,2020-09-20 00:00:00,2020-09-20,2020-09-20,2020-09-20,425
14848,14848,2020-09-20,10,ICE,SNCB/NMBS,2,2022-10-31 20:56:54,2022-10-31 20:56:54,2022-10-31 20:51:00,2022-10-31 20:51:00,ICE: FRANKFURT(MAIN) HBF -> BRUSSEL-ZUID,ans,36,2020-09-20 00:00:00,2020-09-20,2020-09-20,2020-09-20,354
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6358,6358,2020-09-20,19976,IC 19-2,SNCB/NMBS,94,2022-10-31 21:52:36,2022-10-31 21:53:06,2022-10-31 21:50:00,2022-10-31 21:50:00,IC 19-2: TOURNAI -> LILLE FLANDRES,froyennes,94,2020-09-20 00:00:00,2020-09-20,2020-09-20,2020-09-20,156
23009,23009,2020-09-26,19976,IC 19-2,SNCB/NMBS,94,NaT,2022-10-31 21:45:05,NaT,2022-10-31 21:44:00,IC 19-2: TOURNAI -> LILLE FLANDRES,tournai,NaN,NaN,2020-09-26,NaN,2020-09-26,0
23012,23012,2020-09-26,19976,IC 19-2,SNCB/NMBS,94,2022-10-31 21:49:52,2022-10-31 21:50:13,2022-10-31 21:48:00,2022-10-31 21:48:00,IC 19-2: TOURNAI -> LILLE FLANDRES,froyennes,94,2020-09-26 00:00:00,2020-09-26,2020-09-26,2020-09-26,112
23163,23163,2020-09-26,19979,IC 19-2,SNCB/NMBS,94,2022-10-31 07:16:42,2022-10-31 07:17:50,2022-10-31 07:17:00,2022-10-31 07:17:00,IC 19-2: LILLE FLANDRES -> TOURNAI,froyennes,94,2020-09-26 00:00:00,2020-09-26,2020-09-26,2020-09-26,0


In [37]:
unique_trainNrs = full_trips_sorted['Train number'].unique()
autocorr_list = []
for day in range(20,27):
    for number in unique_trainNrs:
        subset = full_trips_sorted[(full_trips_sorted['Train number'] == number) & (full_trips_sorted['Date of departure'].dt.day == day)]
        autocorr_list.append(subset['Delay time'].autocorr(lag = 1))

autocorr_list


AttributeError: Can only use .dt accessor with datetimelike values

In [32]:
full_trips_sorted['Date of departure'].dt.day

AttributeError: Can only use .dt accessor with datetimelike values